In [28]:
import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass

from datetime import datetime 
from datetime import date

@dataclass
class Holiday:
    name: str
    date: datetime.date   
    
    def __str__(self):
        return f'{self.name} ' +  self.date.strftime("%Y-%m-%d")

class HolidayList:
    def __init__(self):
        self.innerHolidays = []

    def addHoliday(self, holidayObj):
        if isinstance(holidayObj, Holiday):
            self.innerHolidays.append(holidayObj)
            print(str(holidayObj) + " has been added to the Holiday Manager")
            
        else:
            print("This is not a holiday object.")
            return
        
        
    def findHoliday(self, HolidayName, HolidayDate):
        for x in self.innerHolidays:
            if x.name == HolidayName and x.date == HolidayDate:
                return Holiday
            else:
                print("That holiday was not found.")
    


    def removeHoliday(self, HolidayName):
        for i in self.innerHolidays:
            if i.name == HolidayName:
                self.innerHolidays.remove(i)
                print(f'{i} has been removed from the holiday manager.')


    def read_json(self, filelocation):
        with open(filelocation, 'r') as f:
            data = json.load(f)
            for i in data["holidays"]:
                dateString = i["date"]
                fDate = datetime.strptime(dateString, "%Y-%m-%d")
                holiday = Holiday(i["name"], fDate)
                self.addHoliday(holiday)

    def save_to_json(self, filelocation):
        with open(filelocation, 'w') as h:
            tmp = []
            for i in self.innerHolidays:
                holiday = {"name": i.name, "date": i.date}
                tmp.append(holiday)
            json.dump(tmp, h, indent = 4, sort_keys = True, default = str)

    def scrapeHolidays(self):
        scraped = []
    
        for year in range(2020,2025):
            html = requests.get(f'https://www.timeanddate.com/holidays/us/{year}?hol=33554809')

            soup = BeautifulSoup(html.text, 'html.parser')
            table = soup.find('table', attrs={'id':'holidays-table'})
            body = table.find('tbody')

            for row in body.find_all('tr'):
                holidayDict = {}

                date = row.find('th')
                name = row.find('a')

                
                if date is not None and name is not None:
                
                    date = date.text
                    date = f"{date} {year}"
                    date= datetime.strptime(date,"%b %d %Y")
                    date = date.strftime('%Y-%m-%d')
                
                    holidayDict['Name'] = name.text
                    holidayDict['Date'] = date
            
                    scraped.append(holidayDict)

                    while {} in scraped:
                        scraped.remove({}) 
                        
                    scraped = [dict(t) for t in {tuple(d.items()) for d in scraped}]

            for i in scraped:
                hol = (Holiday(i['Name'], i['Date']))
                if hol not in self.innerHolidays:
                    self.innerHolidays.append(hol)
        


    def numHolidays(self):
        print(len(self.innerHolidays))

    def filterHolidaysbyWeek(self, year, week):
        results = filter(lambda x: x.date.year == year, filter(lambda x: x.date.isocalendar()[1] == week, self.innerHolidays))
        return results



    def displayHolidaysInWeek(self, holidayList):
        for i in holidayList:
            print(str(i))

    def viewCurrentWeek(self):
         
        current_year = datetime.datetime.now().year
        current_week = datetime.datetime.now().isocalendar().week
        
        holidayList = self.filter_holidays_by_week(current_year, current_week)
        
        self.displayHolidaysInWeek(holidayList)
        

        

def main():

    holidays = HolidayList()
    
    holidays.read_json('holidays.json')

    holidays.scrapeHolidays()
    print('-----------------------------------------------------------------------------------------------------------\n')

    print('Welcome to the Holiday Manager Menu!\n')

    print(f'There are currently {len(holidays.innerHolidays)} holidays in the Holiday Manager.\n')

    print("What would you like to do today?\n 1. Add a Holiday\n 2. Remove a Holiday\n 3. Save Holiday List\n 4. View Holidays\n 5. Exit\n")

    tasks = True
    while tasks == True:
        task_choice = int(input("Choose one of the tasks and input the number associated with the option: "))

        if task_choice == 1:
            print('\n Add a Holiday.\n')
            holidayInput = str(input("Holiday Name: "))
            dateInput = input("Holiday Date (Please input date as YYYY-MM-DD): ")
            dateInput = datetime.strptime(dateInput,"%Y-%m-%d")
            holidays.addHoliday(Holiday(holidayInput, dateInput))
            
        elif task_choice == 2:
            print('\n Remove a Holiday.\n')
            holidayRemove = str(input('Enter the name of the holiday you wish to remove: '))
            # dateRemove = input('Enter the date of the holiday you wish to remove (YYYY-MM-DD): ')
            # dateRemove = datetime.datetime.strptime(dateRemove,"%Y-%m-%d")
            holidays.removeHoliday(holidayRemove)
            
        elif task_choice == 3:
            print("\n Save Holiday List. \n")
            wantSave = str(input("Save your changes? [Y/N]: "))
            if wantSave == "Y":
                
                holidays.save_to_json('final_holidays.json')
                print("Success! Your changes have been saved!")
                savedWork = True
            else:
                print("Changes were not saved.")

                
        elif task_choice == 4:
            print("\n View Holidays. \n")
            year = int(input("Which year?: "))
            week = int(input("Which week? [1-52, 0 for current week]: "))
            if week == 0 or year == 0:
                holidays.viewCurrentWeek()
            else:
                print(f"These are the holidays for year {year}, week {week}:")
                holidays.displayHolidaysInWeek(holidays.filterHolidaysbyWeek(year, week))

            
        elif task_choice == 5:
            exitInput = input("Are you sure you wish to exit? (Y/N) ")
            if exitInput.upper == "Y":
                print("You will now exit the Holiday Manager.")
                tasks = False
                break
            elif exitInput == "N":
                tasks = True 
            else:
                print("Not a valid option. Please enter Y or N.")


        
        
if __name__ == "__main__":
    main();

Margaret Thatcher Day 2021-01-10 has been added to the Holiday Manager
World Sketchnote Day 2021-01-11 has been added to the Holiday Manager
Zanzibar Revolution Day 2021-01-12 has been added to the Holiday Manager
National Rubber Ducky Day 2021-01-13 has been added to the Holiday Manager
Tamil Thai Pongdal Day 2021-01-14 has been added to the Holiday Manager
National Bagel Day 2021-01-15 has been added to the Holiday Manager
Signing of the Peace Accords 2021-01-16 has been added to the Holiday Manager
-----------------------------------------------------------------------------------------------------------

Welcome to the Holiday Manager Menu!

There are currently 2638 holidays in the Holiday Manager.

What would you like to do today?
 1. Add a Holiday
 2. Remove a Holiday
 3. Save Holiday List
 4. View Holidays
 5. Exit


 Add a Holiday.

Halloween 2025-10-31 has been added to the Holiday Manager

 Remove a Holiday.

Margaret Thatcher Day 2021-01-10 has been removed from the holiday 

AttributeError: 'str' object has no attribute 'isocalendar'